# Test

## Matrix Product State

In [13]:
using CUDA
using ITensors, ITensorMPS, Plots

In [14]:
a = 1
ga = 0.5
ma = 0.25
g = ga/a
m_org = ma/a
m_lat = m_org - a*g^2/8
# x = 1/(a^2*g^2)
# mu = 2*m_lat/(a*g^2)

0.21875

In [15]:
N = 10
sites = siteinds("S=1/2", N)

os = OpSum()
for n in 1:N-1
    # os += (1/(4*a)), "X", n,"X", n+1
    # os += (1/(4*a)), "Y", n,"Y", n+1
    os += (1/(2*a)), "S+", n,"S-", n+1
    os += (1/(2*a)), "S+", n+1,"S-", n
end

for n in 1:N
    os += (-1)^n*m_lat/2, "Z", n
end

for n in 1:N-1
    for k in 1:n
        os += g^2*a/4*(-1)^k, "Z", k
        os += g^2*a/4, "Id", 1
    end
    for j in 1:n
        for i in 1:j-1
            os += g^2*a/4, "Z", i, "Z", j
            os += g^2*a/4*(-1)^(i+j), "Id", 1
            os += g^2*a/4*(-1)^j, "Z", i
            os += g^2*a/4*(-1)^i, "Z", j
        end
    end
end

H = cu(MPO(os, sites))

MPO
[1] ((dim=5|id=927|"Link,l=1"), (dim=2|id=434|"S=1/2,Site,n=1")', (dim=2|id=434|"S=1/2,Site,n=1"))
[2] ((dim=5|id=927|"Link,l=1"), (dim=5|id=737|"Link,l=2"), (dim=2|id=399|"S=1/2,Site,n=2")', (dim=2|id=399|"S=1/2,Site,n=2"))
[3] ((dim=5|id=737|"Link,l=2"), (dim=5|id=849|"Link,l=3"), (dim=2|id=103|"S=1/2,Site,n=3")', (dim=2|id=103|"S=1/2,Site,n=3"))
[4] ((dim=5|id=849|"Link,l=3"), (dim=5|id=722|"Link,l=4"), (dim=2|id=612|"S=1/2,Site,n=4")', (dim=2|id=612|"S=1/2,Site,n=4"))
[5] ((dim=5|id=722|"Link,l=4"), (dim=5|id=742|"Link,l=5"), (dim=2|id=288|"S=1/2,Site,n=5")', (dim=2|id=288|"S=1/2,Site,n=5"))
[6] ((dim=5|id=742|"Link,l=5"), (dim=5|id=333|"Link,l=6"), (dim=2|id=112|"S=1/2,Site,n=6")', (dim=2|id=112|"S=1/2,Site,n=6"))
[7] ((dim=5|id=333|"Link,l=6"), (dim=5|id=161|"Link,l=7"), (dim=2|id=884|"S=1/2,Site,n=7")', (dim=2|id=884|"S=1/2,Site,n=7"))
[8] ((dim=5|id=161|"Link,l=7"), (dim=5|id=279|"Link,l=8"), (dim=2|id=987|"S=1/2,Site,n=8")', (dim=2|id=987|"S=1/2,Site,n=8"))
[9] ((dim=5|id=

In [16]:
nsweeps = 30
maxdim = [10,20,100,100,200]
cutoff = [1E-10]

psi0_init = cu(random_mps(sites, linkdims = 10))
# 

MPS
[1] ((dim=2|id=434|"S=1/2,Site,n=1"), (dim=10|id=752|"Link,l=1"))
[2] ((dim=10|id=752|"Link,l=1"), (dim=2|id=399|"S=1/2,Site,n=2"), (dim=10|id=971|"Link,l=2"))
[3] ((dim=10|id=971|"Link,l=2"), (dim=2|id=103|"S=1/2,Site,n=3"), (dim=10|id=275|"Link,l=3"))
[4] ((dim=10|id=275|"Link,l=3"), (dim=2|id=612|"S=1/2,Site,n=4"), (dim=10|id=534|"Link,l=4"))
[5] ((dim=10|id=534|"Link,l=4"), (dim=2|id=288|"S=1/2,Site,n=5"), (dim=10|id=386|"Link,l=5"))
[6] ((dim=10|id=386|"Link,l=5"), (dim=2|id=112|"S=1/2,Site,n=6"), (dim=10|id=599|"Link,l=6"))
[7] ((dim=10|id=599|"Link,l=6"), (dim=2|id=884|"S=1/2,Site,n=7"), (dim=8|id=717|"Link,l=7"))
[8] ((dim=8|id=717|"Link,l=7"), (dim=2|id=987|"S=1/2,Site,n=8"), (dim=4|id=384|"Link,l=8"))
[9] ((dim=4|id=384|"Link,l=8"), (dim=2|id=491|"S=1/2,Site,n=9"), (dim=2|id=935|"Link,l=9"))
[10] ((dim=2|id=935|"Link,l=9"), (dim=2|id=275|"S=1/2,Site,n=10"))


In [17]:
energy0,psi0 = dmrg(H,psi0_init;nsweeps,maxdim,cutoff)

After sweep 1 energy=-3.0984664  maxlinkdim=10 maxerr=2.89E-04 time=0.081
After sweep 2 energy=-3.0987165  maxlinkdim=19 maxerr=1.05E-07 time=0.092
After sweep 3 energy=-3.0987184  maxlinkdim=27 maxerr=5.84E-08 time=0.056
After sweep 4 energy=-3.0987191  maxlinkdim=30 maxerr=4.34E-08 time=0.061
After sweep 5 energy=-3.0987194  maxlinkdim=18 maxerr=2.08E-07 time=0.056
After sweep 6 energy=-3.0987186  maxlinkdim=22 maxerr=9.60E-08 time=0.056
After sweep 7 energy=-3.0987165  maxlinkdim=23 maxerr=4.49E-08 time=0.076
After sweep 8 energy=-3.0987186  maxlinkdim=27 maxerr=1.58E-07 time=0.055
After sweep 9 energy=-3.098719  maxlinkdim=24 maxerr=6.65E-08 time=0.052
After sweep 10 energy=-3.0987182  maxlinkdim=23 maxerr=2.22E-07 time=0.056
After sweep 11 energy=-3.09872  maxlinkdim=26 maxerr=1.42E-07 time=0.048
After sweep 12 energy=-3.0987177  maxlinkdim=23 maxerr=1.13E-07 time=0.048
After sweep 13 energy=-3.0987158  maxlinkdim=24 maxerr=3.27E-08 time=0.065
After sweep 14 energy=-3.0987158  max

(-3.0987184f0, MPS
[1] ((dim=2|id=34|"Link,l=1"), (dim=2|id=434|"S=1/2,Site,n=1"))
[2] ((dim=4|id=303|"Link,l=2"), (dim=2|id=399|"S=1/2,Site,n=2"), (dim=2|id=34|"Link,l=1"))
[3] ((dim=2|id=103|"S=1/2,Site,n=3"), (dim=7|id=242|"Link,l=3"), (dim=4|id=303|"Link,l=2"))
[4] ((dim=2|id=612|"S=1/2,Site,n=4"), (dim=7|id=354|"Link,l=4"), (dim=7|id=242|"Link,l=3"))
[5] ((dim=2|id=288|"S=1/2,Site,n=5"), (dim=28|id=321|"Link,l=5"), (dim=7|id=354|"Link,l=4"))
[6] ((dim=2|id=112|"S=1/2,Site,n=6"), (dim=15|id=490|"Link,l=6"), (dim=28|id=321|"Link,l=5"))
[7] ((dim=2|id=884|"S=1/2,Site,n=7"), (dim=8|id=482|"Link,l=7"), (dim=15|id=490|"Link,l=6"))
[8] ((dim=2|id=987|"S=1/2,Site,n=8"), (dim=4|id=555|"Link,l=8"), (dim=8|id=482|"Link,l=7"))
[9] ((dim=2|id=491|"S=1/2,Site,n=9"), (dim=2|id=813|"Link,l=9"), (dim=4|id=555|"Link,l=8"))
[10] ((dim=2|id=275|"S=1/2,Site,n=10"), (dim=2|id=813|"Link,l=9"))
)

In [19]:
# psi_contracted = contract(psi)

In [24]:
psi1_init = cu(random_mps(sites, linkdims = 10))
emergy1,psi1 = dmrg(H,[psi0],psi1_init;nsweeps,maxdim,cutoff)

After sweep 1 energy=-2.7535646948089076  maxlinkdim=10 maxerr=9.45E-04 time=18.348
After sweep 2 energy=-2.7541750745227915  maxlinkdim=15 maxerr=8.78E-11 time=0.105
After sweep 3 energy=-2.754176051471301  maxlinkdim=13 maxerr=8.41E-11 time=0.100
After sweep 4 energy=-2.754176052427807  maxlinkdim=13 maxerr=9.58E-11 time=0.086
After sweep 5 energy=-2.7541760524274626  maxlinkdim=13 maxerr=7.15E-11 time=0.089
After sweep 6 energy=-2.7541760524274657  maxlinkdim=13 maxerr=7.06E-11 time=0.099
After sweep 7 energy=-2.7541760524274714  maxlinkdim=13 maxerr=7.06E-11 time=0.075
After sweep 8 energy=-2.7541760524274728  maxlinkdim=13 maxerr=7.06E-11 time=0.073
After sweep 9 energy=-2.7541760524274737  maxlinkdim=13 maxerr=7.06E-11 time=0.072
After sweep 10 energy=-2.7541760524274714  maxlinkdim=13 maxerr=7.06E-11 time=0.101
After sweep 11 energy=-2.7541760524274737  maxlinkdim=13 maxerr=7.06E-11 time=0.068
After sweep 12 energy=-2.754176052427469  maxlinkdim=13 maxerr=7.06E-11 time=0.064
Aft

(-2.7541760524274754, MPS
[1] ((dim=2|id=538|"Link,l=1"), (dim=2|id=434|"S=1/2,Site,n=1"))
[2] ((dim=4|id=594|"Link,l=2"), (dim=2|id=399|"S=1/2,Site,n=2"), (dim=2|id=538|"Link,l=1"))
[3] ((dim=2|id=103|"S=1/2,Site,n=3"), (dim=8|id=953|"Link,l=3"), (dim=4|id=594|"Link,l=2"))
[4] ((dim=2|id=612|"S=1/2,Site,n=4"), (dim=11|id=482|"Link,l=4"), (dim=8|id=953|"Link,l=3"))
[5] ((dim=2|id=288|"S=1/2,Site,n=5"), (dim=13|id=233|"Link,l=5"), (dim=11|id=482|"Link,l=4"))
[6] ((dim=2|id=112|"S=1/2,Site,n=6"), (dim=10|id=293|"Link,l=6"), (dim=13|id=233|"Link,l=5"))
[7] ((dim=2|id=884|"S=1/2,Site,n=7"), (dim=8|id=970|"Link,l=7"), (dim=10|id=293|"Link,l=6"))
[8] ((dim=2|id=987|"S=1/2,Site,n=8"), (dim=4|id=856|"Link,l=8"), (dim=8|id=970|"Link,l=7"))
[9] ((dim=2|id=491|"S=1/2,Site,n=9"), (dim=2|id=158|"Link,l=9"), (dim=4|id=856|"Link,l=8"))
[10] ((dim=2|id=275|"S=1/2,Site,n=10"), (dim=2|id=158|"Link,l=9"))
)

## Quantum Number Conserving

In [28]:
using BenchmarkTools

In [31]:
@benchmark siteqnc = siteinds("S=1/2", 5, conserve_qns = true)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.356 μs …  3.766 ms  ┊ GC (min … max):  0.00% … 99.76%
 Time  (median):     4.944 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.682 μs ± 55.998 μs  ┊ GC (mean ± σ):  16.71% ±  1.73%

  ▇▇                ▆█▆▅▅▅▃                                   
  ██▆▂▁▂▂▁▁▁▁▁▁▁▁▁▂▇████████▆▅▄▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  2.36 μs        Histogram: frequency by time         9.6 μs <

 Memory estimate: 13.68 KiB, allocs estimate: 113.

In [40]:
@benchmark siteqnc = cu(siteinds("S=1/2", 5, conserve_qns = true))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   8.400 μs … 503.900 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     10.300 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.563 μs ±  25.775 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂█▇▅▄▂▂▁▁▁                                                  ▁
  ▅███████████▇▇▇▇▇▆▆▆▆▅▆▄▄▄▆▅▆▇▆▆▅▅▅▆▆▅▄▄▄▄▅▅▆▄▄▄▃▄▅▃▅▄▄▄▅▄▄▅ █
  8.4 μs        Histogram: log(frequency) by time      47.6 μs <

 Memory estimate: 13.96 KiB, allocs estimate: 123.

In [39]:
siteqnc = cu(siteinds("S=1/2", 5, conserve_qns = true))
siteqnc

5-element CuArray{Index{Vector{Pair{QN, Int64}}}, 1, CUDA.DeviceMemory}:
 (dim=2|id=40|"S=1/2,Site,n=1") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1
 (dim=2|id=289|"S=1/2,Site,n=2") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1
 (dim=2|id=267|"S=1/2,Site,n=3") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1
 (dim=2|id=263|"S=1/2,Site,n=4") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1
 (dim=2|id=201|"S=1/2,Site,n=5") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1

# PDF of Schwinger Model with Time evolving MPS

In [42]:
function time_evolve(site, N, delta_t)
    gates = ITensor[]
    for j in 1:N-1
        s1 = site[j]
        s2 = site[j+1]

        h_n = 1/(2*a)*op("S+", s1)*op("S-", s2) + 1/(2*a)*op("S+", s2)*op("S-", s1) 
            + (-1)^j*m_lat/2*op("Z", s1) + (-1)^(j+1)*m_lat/2*op("Z", s2)
            + 
        
        for k in 1:j
            h_n += g^2*a/4*(-1)^k*op("Z", k)
            h_n += g^2*a/4*op("Id", 1)
            for i in 1:k-1
                h_n += g^2*a/4*op("Z", i)*op("Z", k)
                h_n += g^2*a/4*(-1)^(i+k)*op("Id", 1)
                h_n += g^2*a/4*(-1)^k*op("Z", i)
                h_n += g^2*a/4*(-1)^i*op("Z", k)
            end
        end
        G_j = exp(-im*h_n*delta_t)
        push!(gates, G_j)
    end
end
    

time_evolve (generic function with 2 methods)